## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,55.40,67,20,37.98,few clouds
1,1,Saint-Augustin,CA,51.2260,-58.6502,34.50,96,98,3.38,overcast clouds
2,2,Jamestown,US,42.0970,-79.2353,35.60,48,1,8.05,clear sky
3,3,Soyo,AO,-6.1349,12.3689,81.05,84,100,8.75,overcast clouds
4,4,Provideniya,RU,64.3833,-173.3000,-9.94,87,100,28.03,light snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Soyo,AO,-6.1349,12.3689,81.05,84,100,8.75,overcast clouds
5,5,Vaini,TO,-21.2000,-175.2000,80.60,94,75,9.22,light intensity drizzle
7,7,Fayaoue,NC,-20.6500,166.5333,80.53,82,8,17.94,clear sky
16,16,Atuona,PF,-9.8000,-139.0333,78.60,69,71,10.92,broken clouds
19,19,Geraldton,AU,-28.7667,114.6000,75.20,31,40,24.16,scattered clouds
23,23,Saint-Philippe,RE,-21.3585,55.7679,80.60,69,20,3.44,light rain
24,24,Isangel,VU,-19.5500,169.2667,77.00,83,40,5.75,scattered clouds
28,28,Mahebourg,MU,-20.4081,57.7000,78.80,78,40,11.50,light rain
29,29,Caravelas,BR,-17.7125,-39.2481,79.38,75,1,8.59,clear sky
41,41,Tual,ID,-5.6667,132.7500,79.56,88,100,32.75,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                186
City                   186
Country                185
Lat                    186
Lng                    186
Max Temp               186
Humidity               186
Cloudiness             186
Wind Speed             186
Current Description    186
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                185
City                   185
Country                185
Lat                    185
Lng                    185
Max Temp               185
Humidity               185
Cloudiness             185
Wind Speed             185
Current Description    185
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Soyo,AO,81.05,overcast clouds,-6.1349,12.3689,
5,Vaini,TO,80.60,light intensity drizzle,-21.2000,-175.2000,
7,Fayaoue,NC,80.53,clear sky,-20.6500,166.5333,
16,Atuona,PF,78.60,broken clouds,-9.8000,-139.0333,
19,Geraldton,AU,75.20,scattered clouds,-28.7667,114.6000,
23,Saint-Philippe,RE,80.60,light rain,-21.3585,55.7679,
24,Isangel,VU,77.00,scattered clouds,-19.5500,169.2667,
28,Mahebourg,MU,78.80,light rain,-20.4081,57.7000,
29,Caravelas,BR,79.38,clear sky,-17.7125,-39.2481,
41,Tual,ID,79.56,overcast clouds,-5.6667,132.7500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] =f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Cannot find hotel...skipping.")

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Soyo,AO,81.05,overcast clouds,-6.1349,12.3689,Hotel Mpampa
5,Vaini,TO,80.60,light intensity drizzle,-21.2000,-175.2000,Keleti Beach Resort
7,Fayaoue,NC,80.53,clear sky,-20.6500,166.5333,Chez Dydyce Guesthouse
16,Atuona,PF,78.60,broken clouds,-9.8000,-139.0333,Villa Enata
19,Geraldton,AU,75.20,scattered clouds,-28.7667,114.6000,Broadwater Mariner Resort
23,Saint-Philippe,RE,80.60,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
24,Isangel,VU,77.00,scattered clouds,-19.5500,169.2667,Volcano Sea View Tree House
28,Mahebourg,MU,78.80,light rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
29,Caravelas,BR,79.38,clear sky,-17.7125,-39.2481,Pousada dos Navegantes
41,Tual,ID,79.56,overcast clouds,-5.6667,132.7500,Grand Vilia Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))